# processor

> Fill in a module description here

In [ ]:
#| default_exp processor

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from typing import List, Dict, Any, Optional, Union
from datetime import datetime
from llm_data_extractor.models import ExtractionResult
from llm_data_extractor.models import ExtractionResult, LLMConfig, DBConfig, Question
import pandas as pd
from llm_data_extractor.db import get_connection, fetch_data, insert_results
import concurrent.futures
from llm_data_extractor.extract import extract_data
import logging
import ast
from llm_data_extractor.log_utils import setup_logger, call_with_visible_output
from tqdm import tqdm

In [ ]:
#| export

log = setup_logger("processor")  # call once per process

In [ ]:
#| export

def format_for_db(results: List[ExtractionResult], 
                  source_id: Optional[str] = None,
                  batch_id: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    Format extraction results for database insertion.
    
    Args:
        results: List of ExtractionResult objects
        source_id: Optional identifier for the source document/text
        batch_id: Optional identifier for the processing batch
        
    Returns:
        List of dictionaries ready for database insertion
    """
    
    formatted_results = []
    
    for result in results:
        record = {
            'question_id': result.question_id,
            'raw_answer': result.raw_answer,
            'parsed_answer': _serialize_answer(result.parsed_answer),
            'confidence': result.confidence,
            'is_valid': result.is_valid,
            'validation_error': result.validation_error,
            'processed_at': result.timestamp.isoformat(),
            'source_id': source_id,
            'batch_id': batch_id
        }
        
        formatted_results.append(record)
    
    return formatted_results


def format_for_target_tables(results: List[ExtractionResult], 
                           questions_map: Dict[str, Any]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Format results grouped by target table for direct insertion into business tables.
    
    Args:
        results: List of ExtractionResult objects
        questions_map: Map of question_id to Question objects
        
    Returns:
        Dictionary with table names as keys and records as values
    """
    
    table_records = {}
    
    for result in results:
        if not result.is_valid:
            continue  # Skip invalid results
            
        question = questions_map.get(result.question_id)
        if not question:
            continue
            
        table_name = question.target_table
        field_name = question.target_field
        
        if table_name not in table_records:
            table_records[table_name] = {}
        
        # Use source_id or batch_id as the record key
        # This assumes one record per source document per table
        record_key = 'default'  # You might want to pass this in
        
        if record_key not in table_records[table_name]:
            table_records[table_name][record_key] = {}
        
        table_records[table_name][record_key][field_name] = result.parsed_answer
        table_records[table_name][record_key][f'{field_name}_confidence'] = result.confidence
        table_records[table_name][record_key][f'{field_name}_processed_at'] = result.timestamp.isoformat()
    
    # Convert to list format for each table
    formatted_tables = {}
    for table_name, records in table_records.items():
        formatted_tables[table_name] = list(records.values())
    
    return formatted_tables


def create_summary_stats(results: List[ExtractionResult]) -> Dict[str, Any]:
    """
    Create summary statistics for a batch of extraction results.
    
    Args:
        results: List of ExtractionResult objects
        
    Returns:
        Dictionary with summary statistics
    """
    
    if not results:
        return {
            'total_questions': 0,
            'valid_answers': 0,
            'invalid_answers': 0,
            'success_rate': 0.0,
            'avg_confidence': 0.0
        }
    
    total_questions = len(results)
    valid_answers = sum(1 for r in results if r.is_valid)
    invalid_answers = total_questions - valid_answers
    success_rate = valid_answers / total_questions
    
    # Calculate average confidence for valid answers
    valid_confidences = [r.confidence for r in results if r.is_valid and r.confidence is not None]
    avg_confidence = sum(valid_confidences) / len(valid_confidences) if valid_confidences else 0.0
    
    # Get validation error categories
    error_types = {}
    for result in results:
        if not result.is_valid and result.validation_error:
            error_type = result.validation_error.split(':')[0]  # Get first part of error
            error_types[error_type] = error_types.get(error_type, 0) + 1
    
    return {
        'total_questions': total_questions,
        'valid_answers': valid_answers,
        'invalid_answers': invalid_answers,
        'success_rate': round(success_rate, 3),
        'avg_confidence': round(avg_confidence, 3),
        'error_types': error_types,
        'processed_at': datetime.now().isoformat()
    }


def _serialize_answer(answer: Any) -> Union[str,None]:
    """
    Serialize parsed answer for database storage.
    
    Args:
        answer: The parsed answer value
        
    Returns:
        String representation suitable for database storage
    """
    
    if answer is None:      return None
    
    # Handle datetime objects
    if isinstance(answer, datetime):
        return answer.isoformat()
    
    # Handle lists and complex objects
    if isinstance(answer, (list, dict)):
        return json.dumps(answer)
    
    # Handle primitive types
    return str(answer)


def create_audit_record(batch_id: str, 
                       source_id: str,
                       total_questions: int,
                       processing_time_seconds: float,
                       llm_config: Dict[str, Any],
                       summary_stats: Dict[str, Any]) -> Dict[str, Any]:
    """
    Create an audit record for the extraction batch.
    
    Args:
        batch_id: Unique identifier for the batch
        source_id: Identifier for the source document
        total_questions: Number of questions processed
        processing_time_seconds: Time taken for processing
        llm_config: Configuration used for LLM calls
        summary_stats: Summary statistics from create_summary_stats
        
    Returns:
        Audit record dictionary
    """
    
    return {
        'batch_id': batch_id,
        'source_id': source_id,
        'total_questions': total_questions,
        'processing_time_seconds': round(processing_time_seconds, 2),
        'llm_model': llm_config.get('model_name'),
        'llm_temperature': llm_config.get('temperature'),
        'success_rate': summary_stats.get('success_rate'),
        'avg_confidence': summary_stats.get('avg_confidence'),
        'processed_at': datetime.now().isoformat(),
        'config_snapshot': json.dumps(llm_config)
    }

In [ ]:
#| export

REQUIRED_COLUMNS = [
    'input_row_id', 'input_text', 'question_id', 'question_text', 
    'answer_type', 'answer_config', 'table_name', 'field_name'
]


def _process_batch(batch_info: Dict[str, Any], llm_config) -> List:
    log.debug("_process_batch() called")
    log.debug("batch_info type: %s", type(batch_info))
    results, metadata = call_with_visible_output(
        extract_data,
        source_text=batch_info['input_text'],
        questions=batch_info['questions'],
        llm_config=llm_config,
        source_id=batch_info['input_row_id'],
        batch_id=batch_info['input_row_id'],
    )
    log.debug("Results from extract_data(): %s", results)
    log.debug("Metadata from extract_data(): %s", metadata)
    for r in results:
        r.table_name = batch_info['table_name']
        r.field_name = batch_info['field_name']
        r.input_row_id = batch_info['input_row_id']
    return results

def process_query(query: str, llm_config: LLMConfig, db_config: DBConfig, results_table_name: str, max_workers: int = 4) -> Dict[str, Any]:
    """
    Fetches data from Snowflake, processes it in parallel using an LLM, and inserts results back.

    Args:
        query: SQL query to fetch the data to be processed.
        llm_config: Configuration for the LLM.
        db_config: Configuration for the database connection.
        results_table_name: Name of the table to store the results.
        max_workers: The maximum number of threads to use for parallel processing.

    Returns:
        A dictionary containing summary statistics of the processing run.
    """
    log.debug('Establishing Connection')
    # 1. Fetch data
    conn = get_connection(db_config)
    log.debug('Getting data')
    try:
        df = fetch_data(query, conn)
    finally:
        conn.close()

    # 2. Validate columns
    if not all(col.upper() in [c.upper() for c in df.columns] for col in REQUIRED_COLUMNS):
        missing = set(REQUIRED_COLUMNS) - set(df.columns)
        raise ValueError(f"Query result is missing required columns: {missing}")
    
    # Normalize column names to lowercase
    df.columns = [col.lower() for col in df.columns]
    log.debug(f'Columns: {df.columns}')

    # 3. Group data into batches

    def parse_config(x):
        if isinstance(x, dict):
            return x
        if isinstance(x, str):
            try:
                return json.loads(x)  # works if it's valid JSON
            except json.JSONDecodeError:
                return ast.literal_eval(x)  # fallback for Python-style strings
        return {}
    log.debug('Group data into batches')
    df['answer_config'] = df['answer_config'].apply(parse_config)

    # Now this 
    # s a dict already
    log.debug(df['answer_config'].iloc[0])
    
    batches = []
    for group_name, group_df in df.groupby('input_row_id'):
        questions = [
            Question(
                id=str(row['question_id']),
                text=row['question_text'],
                answer_type=row['answer_type'],
                answer_config=row['answer_config'] or {}   # already a dict
            ) for _, row in group_df.iterrows()
        ]
        
        batch_info = {
            'input_row_id': group_name,                     #input_row_id value
            'input_text': group_df['input_text'].iloc[0],
            'table_name': group_df['table_name'].iloc[0],
            'field_name': group_df['field_name'].iloc[0],
            'questions': questions
        }
        batches.append(batch_info)

    log.debug(f'Batches: {batches}')
    # 4. Process batches in parallel
    all_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {executor.submit(_process_batch, batch, llm_config): batch for batch in batches}
        for future in tqdm(concurrent.futures.as_completed(future_to_batch), total=len(batches), desc="Processing Batches"):
            try:
                results = future.result()
                log.debug(f'Result: {results}')
                all_results.extend(results)
            except Exception as exc:
                batch_info = future_to_batch[future]
                log.debug(f"Batch for input_row_id {batch_info['input_row_id']} generated an exception: {exc}")

    # 5. Insert results into DB
    if all_results:
        conn = get_connection(db_config)
        try:
            db_records = []
            for res in all_results:
                record = {
                    "TABLE_NAME": res.table_name,
                    "FIELD_NAME": res.field_name,
                    "FIELD_ID": res.input_row_id,
                    "QUESTION_ID": res.question_id,
                    "results": json.dumps({
                        "raw_answer": res.raw_answer, 
                        "parsed_answer": _serialize_answer(res.parsed_answer), 
                        "is_valid": res.is_valid, 
                        "confidence": res.confidence,
                        "validation_error": res.validation_error
                    })
                }
                db_records.append(record)
            
            insert_results(db_records, conn, results_table_name)
        finally:
            conn.close()

    # 6. Return summary
    summary = create_summary_stats(all_results)
    return summary

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()